In [1]:
import pandas as pd
import numpy as np
import difflib

In [2]:
cd C:\Users\s153529\PycharmProjects\european_vote_analysis

C:\Users\s153529\PycharmProjects\european_vote_analysis


In [97]:
data_european = pd.read_csv('data/raw/COMEPELDA_aggregate_v1.00.csv')
data_national = pd.read_csv('data/raw/1999-2019_CHES_dataset_means.csv')
country_codes = pd.read_csv('data/raw/countries.csv')  
party_names = pd.read_csv('data/raw/partyNames_ncv3.csv')  

In [98]:
# Add country codes to national data
data_european = pd.merge(data_european, country_codes, left_on='cName', right_on='Country', how='left')
# drop 2006, 2007, 2013 years due to particular countries elections
data_european = data_european.drop(data_european[data_european['ElYear'].isin([2006, 2007, 2013])].index)
data_european.reset_index(drop=True, inplace=True)

In [99]:
data_european.head()

,IDAD,cName,ElYear,IDpty,pName,dName,dNameEng,cValid,dValid,pcAbs,...,pAddLisbon,pAlliance01,IDauxAlliance,pApparParty1,pApparParty2,IDauxElec,IDauxIndep,ID,Abbreviation,Country
0,1,Austria,1999,50,Freiheitliche Partei Österreichs,(nationwide),(nationwide),2801353,2801353,655519,...,0,0,.,.,.,16,.,13,AUS,Austria
1,2,Austria,1999,955,Liberales Forum,(nationwide),(nationwide),2801353,2801353,74467,...,0,0,.,.,.,16,.,13,AUS,Austria
2,3,Austria,1999,973,Sozialdemokratische Partei Österreichs,(nationwide),(nationwide),2801353,2801353,888338,...,0,0,.,.,.,16,.,13,AUS,Austria
3,4,Austria,1999,1013,Österreichische Volkspartei,(nationwide),(nationwide),2801353,2801353,859175,...,0,0,.,.,.,16,.,13,AUS,Austria
4,5,Austria,1999,1429,Die Grünen – Die Grüne Alternative,(nationwide),(nationwide),2801353,2801353,260273,...,0,0,.,.,.,16,.,13,AUS,Austria


In [100]:
# add country codes to european data
data_national = pd.merge(data_national, country_codes, left_on='country', right_on='ID', how='left')

In [101]:
party_names.head()

,CountryAbrev;partyID;partyAbrev;partyName;partyNameEng
0,BE;102;PS;Parti Socialiste;Socialist Party
1,BE;103;SP;Socialistische Partij;Socialist Party
2,BE;103;SPA;Socialistische Partij Anders;Social...
3,BE;104;ECOLO;Ecolo;Ecolo
4,BE;105;AGALEV;Agalev;Agalev


In [92]:
data_national = pd.merge(data_national, party_names, left_on='party_id', right_on='partyID', how='left')

KeyError: 'partyID'

In [94]:
data_national.head()

,country,eastwest,eumember,year,expert,party_id,cmp_id,party,vote,seat,...,antielite_salience,corrupt_salience,members_vs_leadership,mip_one,mip_two,mip_three,chesversion,ID,Abbreviation,Country
0,1,1,1,1999,9,115,NaN,FN,1.5,0.7,...,NaN,NaN,NaN,NaN,NaN,NaN,2020.1,1,BE,Belgium
1,1,1,1,1999,9,109,21521.0,CVP,14.1,14.7,...,NaN,NaN,NaN,NaN,NaN,NaN,2020.1,1,BE,Belgium
2,1,1,1,1999,9,107,21421.0,PVV/VLD,14.3,15.3,...,NaN,NaN,NaN,NaN,NaN,NaN,2020.1,1,BE,Belgium
3,1,1,1,1999,9,106,21422.0,PRL,7.7,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2020.1,1,BE,Belgium
4,1,1,1,1999,9,110,21913.0,VU,5.6,5.3,...,NaN,NaN,NaN,NaN,NaN,NaN,2020.1,1,BE,Belgium


In [93]:
test_data = pd.merge(data_national, data_european, left_on=["Country", 'partyName'], right_on=["cName", 'pName'], how='outer', indicator=True)

KeyError: 'partyName'

In [8]:
T1 = test_data[test_data['_merge'] == "left_only"]
T2 = test_data[test_data['_merge'] == "right_only"]
T3 = test_data[test_data['_merge'] == "both"]

In [42]:
year_corr = {1999: 1999, 
 2004: 2002,
 2009: 2006,
 2014: 2014}

In [83]:
data_european["alt_partyName"] = [difflib.get_close_matches(data_european.loc[idx, "pName"], data_national[(data_european.loc[idx, "cName"] == data_national["Country"]) & (year_corr[data_european.loc[idx, "ElYear"]] == data_national["year"])]["partyName"].astype(str), n=1, cutoff=0.8) for idx in range(len(data_european))]

In [84]:

data_european["alt_PNCert"] = [difflib.SequenceMatcher(None, data_european.loc[idx,"pName"], data_european.loc[idx, "alt_partyName"][0]).ratio() if len(data_european.loc[idx, "alt_partyName"]) > 0 else 0 for idx in range(len(data_european))]

In [85]:
data_european[["cName","ElYear", "pName", "alt_partyName", "alt_PNCert"]]

,cName,ElYear,pName,alt_partyName,alt_PNCert
0,Austria,1999,Freiheitliche Partei Österreichs,[Freiheitliche Partei Österreichs],1.000000
1,Austria,1999,Liberales Forum,[Liberales Forum],1.000000
2,Austria,1999,Sozialdemokratische Partei Österreichs,[Sozialdemokratische Partei Österreichs],1.000000
3,Austria,1999,Österreichische Volkspartei,[Österreichische Volkspartei],1.000000
4,Austria,1999,Die Grünen – Die Grüne Alternative,[],0.000000
...,...,...,...,...,...
1769,Sweden,2014,Socialdemokraterna,[],0.000000
1770,Sweden,2014,Miljöpartiet de Gröna,[Miljöepartiet de Gröna],0.976744
1771,Sweden,2014,Centerpartiet,[Centerpartiet],1.000000
1772,Sweden,2014,Feministiskt initiativ,[Feministiskt initiativ],1.000000


In [87]:
data_national[(data_national["Country"] == "Great Britain") & (data_national["year"] == 2014)]

,country,eastwest,eumember,year,expert,party_id,cmp_id,party,vote,seat,...,mip_three,chesversion,ID,Abbreviation,Country,CountryAbrev,partyID,partyAbrev,partyName,partyNameEng
749,11,1,1,2014,7,1107,NaN,GREEN,1.0,0.200000,...,2.0,2020.1,11,UK,Great Britain,UK,1107.0,Green,Green Party,Green Party
752,11,1,1,2014,7,1106,51901.0,PLAID,0.6,0.500000,...,17.0,2020.1,11,UK,Great Britain,UK,1106.0,Plaid,Plaid Cymru,Party of Wales
755,11,1,1,2014,7,1108,51951.0,UKIP,3.1,0.000000,...,1.0,2020.1,11,UK,Great Britain,UK,1108.0,UKIP,United Kingdom Independence Party,United Kingdom Independence Party
758,11,1,1,2014,7,1105,51902.0,SNP,1.7,0.900000,...,13.0,2020.1,11,UK,Great Britain,UK,1105.0,SNP,Scottish National Party,Scottish National Party
761,11,1,1,2014,7,1102,51320.0,LAB,29.0,39.700001,...,9.0,2020.1,11,UK,Great Britain,UK,1102.0,Lab,Labour Party,Labour Party
764,11,1,1,2014,7,1104,51421.0,LibDem,23.0,8.800000,...,27.0,2020.1,11,UK,Great Britain,UK,1104.0,LibDem,Liberal Democratic Party,Liberal Democratic Party
767,11,1,1,2014,7,1101,51620.0,CONS,36.1,47.200001,...,5.0,2020.1,11,UK,Great Britain,UK,1101.0,Cons,Conservative Party,Conservative Party


In [ ]:
data_european[["Country", "partyName", "alt_partyName"]]